In [4]:
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import AdamW

import numpy as np

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
import pytorch_lightning as pl

from transformers import AutoTokenizer
from transformers import GPT2Tokenizer
from transformers import get_linear_schedule_with_warmup
from transformers import GPT2LMHeadModel
from datasets import load_dataset




## Data

In [5]:
class XSumPreprocessor:
    def __init__(self, tokenizer, max_input_length, max_target_length, prefix='summarize'):
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
        self.prefix = prefix
  
    def preprocess(self, examples):
        # encode the code-docstring pairs
        texts = examples['document']
        summaries = examples['summary']
        
        inputs = [self.prefix + text for text in texts]
        model_inputs = self.tokenizer(inputs, max_length=self.max_input_length, padding="max_length", truncation=True)

        # encode the summaries
        labels = self.tokenizer(summaries, max_length=self.max_target_length, padding="max_length", truncation=True).input_ids

        # important: we need to replace the index of the padding tokens by -100
        # such that they are not taken into account by the CrossEntropyLoss
        labels_with_ignore_index = []
        for labels_example in labels:
            labels_example = [label if label != 0 else -100 for label in labels_example]
            labels_with_ignore_index.append(labels_example)
        
        model_inputs["labels"] = labels_with_ignore_index

        return model_inputs
    
dataset = load_dataset("xsum")
print(dataset['train'][0])
  

Found cached dataset xsum (/home/reza/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [8]:
max_input_length = 512
max_target_length = 512
prefix = "summarize"
model_name = "gpt2"

# tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

preprocessor = XSumPreprocessor(tokenizer=tokenizer,
                                max_input_length=max_input_length,
                                max_target_length=max_target_length,
                                prefix=prefix)
processor = preprocessor.preprocess
dataset = dataset.map(processor, batched=True)

## Model

In [ ]:
class GPT2PreTrained(pl.LightningModule):
    def __init__(self, lr=5e-5, max_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = GPT2LMHeadModel.from_pretrained("gpt2")
        self.save_hyperparameters()
        self.val_dataloader_ = None
        self.test_dataloader_ = None
        self.train_dataloader_ = None

    def forward(self, input_ids, labels, attention_mask):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.model.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        training_steps = self.hparams.max_epochs * len(self.train_dataloader_)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=training_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def set_train_dataloader(self, train_dataloader):
        self.train_dataloader_ = train_dataloader

    def set_valid_dataloader(self, valid_dataloader):
        self.val_dataloader_ = valid_dataloader

    def set_test_dataloader(self, test_dataloader):
        self.test_dataloader_ = test_dataloader

    def train_dataloader(self):
        return self.train_dataloader_
    
    def val_dataloader(self):
        return self.val_dataloader_
    
    def test_dataloader(self):
        return self.test_dataloader_


## Training

In [ ]:
# hyperparameters
batch_size = 1000
learning_rate = 5e-5
max_epochs = 10

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

train_dataloader = DataLoader(dataset['train[:10%]'], shuffle=True, batch_size=batch_size)
valid_dataloader = DataLoader(dataset['validation'], batch_size=batch_size)
test_dataloader = DataLoader(dataset['test'], batch_size=batch_size)

early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor()
checkpoint_callback = ModelCheckpoint(dirpath='/saved/models/GPT2Summerizer', monitor='validation_loss', mode='min')

trainer = Trainer(accelerator="gpu", 
                  #default_root_dir="/content/drive/MyDrive/T5/Notebooks/Checkpoints", 
                  callbacks=[early_stop_callback, lr_monitor, checkpoint_callback])


model = GPT2PreTrained(lr=learning_rate, max_epochs=max_epochs)
model.set_train_dataloader(train_dataloader)
model.set_valid_dataloader(valid_dataloader)
model.set_test_dataloader(test_dataloader)

trainer.fit(model)

NameError: name 'dataset' is not defined